<a href="https://colab.research.google.com/github/NnamdiOdozi/Intern_GenAI_Examples/blob/main/main_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Force IPython to auto-flush outputs
%config Application.log_level = 'INFO'
%config InteractiveShell.ast_node_interactivity = "all"


In [1]:
%pip install --upgrade --quiet langchain langchain-core langchain-openai openai tiktoken


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.4/948.4 kB 15.6 MB/s eta 0:00:00


In [2]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

import sys
import os
import io


In [ ]:
# Load environment variables
# from dotenv import load_dotenv
# load_dotenv()

[ColabKernelApp] Aborting c2a8f7c0-f22d-4018-f322-532ed2281138: execute_request


In [3]:
# Used to securely store your API key
from google.colab import userdata

# Access the API key from Colab secrets
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")

# You can now use the OPENAI_API_KEY variable to configure your OpenAI client.
# For example:
# client = OpenAI(api_key=OPENAI_API_KEY)

In [5]:
# Single LLM Call
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0.1,
    api_key= OPENAI_API_KEY,
    request_timeout=120,
    max_retries=5,
    verbose=False,

)

messages = [
    ("assistant", "You are a helpful assistant."),
    ("human", "Who is the president of the USA.")
]



res = llm.invoke(messages)   # list-of-conversations -> ChatResult

print(res.content)

print(res)

As of June 2024, the president of the United States is Joe Biden.
content='As of June 2024, the president of the United States is Joe Biden.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 25, 'total_tokens': 42, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6d7dcc9a98', 'id': 'chatcmpl-CIgtDmN5EeIazT43X2cLtnCVTdfUE', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--31e72d4d-2ee4-4e84-a6bf-c49fd3179576-0' usage_metadata={'input_tokens': 25, 'output_tokens': 17, 'total_tokens': 42, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [ ]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg.content

"J'aime la programmation."

In [10]:
if True:
    # Batched LLM Calls
    from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

    messages = [
        ("assistant", "You are a helpful assistant."),
        ([HumanMessage("Write me a haiku about autumn leaves.")])
    ] * 5

    completion = llm.batch(messages)  # list-of-list-of-conversations -> ChatResult

    print(completion)

    for i, choice in enumerate(completion):
        print(f"\n\nCHOICE {i+1}\n")
        print(choice.content)


[AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 18, 'total_tokens': 27, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4fce0778af', 'id': 'chatcmpl-CIh5XP9aayLEE5VAMsFmuqJmMoxKj', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--f79673fa-31b3-42e8-8585-7cf2a8eac67f-0', usage_metadata={'input_tokens': 18, 'output_tokens': 9, 'total_tokens': 27, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), AIMessage(content='Crimson leaves descend,  \nWhispers of the cooling breeze,  \nAutumn’s soft farewell.', additional_kwargs={'refusal': None}, response_metadata={'t

In [8]:
# Using template allows use insert parameters into the prompt - you only need to provide parameter values at query time
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
res = chain.invoke(
    {
        "input_language": "English",
        "output_language": "French",
        "input": "I love programming.",
    }
)

print(res.content)

J'aime la programmation.


In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# LLM
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.0)

# --- First prompt: reformulate ---
reformulate_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that rewrites questions in a clearer, more formal way."),
    ("user", "Rewrite the following question to be clear and formal: {raw_question}")
])

# --- Second prompt: answer the reformulated question ---
answer_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a knowledgeable tutor."),
    ("user", "Answer this question in a concise and friendly way:\n\n{clean_question}")
])

# --- Chain them together ---
# reformulate → llm → inject into second prompt → llm
chain = (
    reformulate_prompt
    | llm
    | (lambda msg: {"clean_question": msg.content})  # extract content
    | answer_prompt
    | llm
)

# --- Run the chain ---
query = "hey what is chatprompttemplate used for???"
response = chain.invoke({"raw_question": query})

print("\n Input prompt:", query)
print(response.content)

# Run only the first prompt + LLM
first_result = (reformulate_prompt | llm).invoke({"raw_question": query})
print("\n Before reformulation:", query)
print("\n After reformulation:", first_result.content)

second_result = (answer_prompt | llm).invoke({"clean_question": first_result.content})
print("\n After answering:", second_result.content)


 Input prompt: hey what is chatprompttemplate used for???
Sure! ChatPromptTemplate is a tool used to create structured prompts for chat-based AI models. It helps you organize and format the input messages clearly, making it easier to guide the AI’s responses. By using it, you can define roles (like user or assistant), insert variables, and build dynamic conversations efficiently. It’s great for crafting consistent and effective prompts in chat applications!

 Before reformulation: hey what is chatprompttemplate used for???

 After reformulation: Could you please explain the purpose and use of ChatPromptTemplate?

 After answering: Sure! ChatPromptTemplate is a tool used to create structured prompts for chat-based AI models. It helps you organize and format the input messages clearly, making it easier to guide the AI’s responses. By using it, you can define roles (like user or assistant), insert variables, and build dynamic conversations efficiently. It’s great for crafting consistent 

In [12]:
if True:

    # how to have an interactive real time conversation with the assistant?

    # Initialise model
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0, api_key=OPENAI_API_KEY)

    def ask_once(user_input: str):
        # Only system + user, no history
        messages = [
            SystemMessage(content="You are a helpful assistant."),
            HumanMessage(content=user_input)
        ]
        response = llm.invoke(messages)
        return response.content

    # Run a simple q&a session
    i=0
    while True:
        user_input = input(f"Q{i+1}: ")   # prompt user in Jupyter cell
        if user_input == "quit" or user_input == "exit" or user_input == "break" or not user_input.strip():        # allow empty string or quit or exit or break to break early
            break
        answer = ask_once(user_input)
        i += 1
        print(f"Assistant: {answer}\n")

        # Example questions are: where were the last olympics held?  who won the world cup in 2022?  Who wa the team captain? what is the capital of france? who is the president of the united states? what is the tallest mountain in the world?

Q1: who won the world cup in 2022? 
Assistant: Argentina won the 2022 FIFA World Cup. They defeated France in the final, which took place on December 18, 2022, in Qatar. The match ended in a dramatic penalty shootout after a 3-3 draw in regulation and extra time.

Q2: who was the top scorer in the competition?
Assistant: Could you please specify which competition you are referring to? There are many sports and events with top scorers, so I need more details to provide an accurate answer.

Q3: what is the capital of France?
Assistant: The capital of France is Paris.

Q4: 


What did you notice about this Assistant?

In [13]:
if True:

    # What is different about this version?

    # initialise model
    #llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

    # start with a system prompt
    history = [SystemMessage(content="You are a helpful assistant.")]

    # function to ask user input and continue the conversation
    def chat_turn(user_input: str):
        # append user message
        history.append(HumanMessage(content=user_input))

        # call model with full history
        ai_msg = llm.invoke(history)  # returns an AIMessage
        #print(f"Assistant: {ai_msg.content}")

        # append assistant reply manually (so it's fed into the next turn)
        history.append(ai_msg)
        return ai_msg.content

    # Run a simple q&a session
    i=0
    while True:
        user_input = input(f"Q{i+1}: ")   # prompt user in Jupyter cell
        if user_input == "quit" or user_input == "exit" or user_input == "break" or not user_input.strip():        # allow empty string or quit or exit or break to break early
            break
        answer = chat_turn(user_input)
        i += 1
        print(f"Assistant: {answer}\n")

        # Example questions are: where were the last olympics held?  who won the world cup in 2022?  Who wa the team captain? what is the capital of france? who is the president of the united states? what is the tallest mountain in the world?

Q1: who won the world cup in 2022? 
Assistant: Argentina won the 2022 FIFA World Cup. They defeated France in the final, which took place on December 18, 2022, in Qatar. The match ended in a 3-3 draw after extra time, and Argentina won 4-2 in the penalty shootout.

Q2: who was the top scorer in the competition?
Assistant: The top scorer of the 2022 FIFA World Cup was Kylian Mbappé of France. He scored a total of 8 goals during the tournament, including a hat-trick in the final against Argentina.

Q3: who was best player of the tournament?
Assistant: The best player of the 2022 FIFA World Cup, awarded the Golden Ball, was Lionel Messi of Argentina. He played a crucial role in leading his team to victory and had an outstanding tournament, scoring 7 goals and providing key assists.

Q4: who was the 2nd highest goal scorer?
Assistant: The second highest goal scorer of the 2022 FIFA World Cup was Lionel Messi, who scored 7 goals during the tournament. Kylian Mbappé was the top scorer with 8

In [14]:
if True:
    # And about this version?

    # initialise model
    #llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0, output_version="responses/v1")

    tool = {"type": "web_search_preview"}
    llm_with_tools = llm.bind_tools([tool])

    # start with a system prompt
    history = [SystemMessage(content="You are a helpful assistant.")]

    # function to ask user input and continue the conversation
    def chat_turn(user_input: str):
        # append user message
        history.append(HumanMessage(content=user_input))

        # call model with full history
        ai_msg = llm_with_tools.invoke(history)  # returns an AIMessage
        #print(f"Assistant: {ai_msg.content}")

        # append assistant reply manually (so it's fed into the next turn)
        history.append(ai_msg)
        return ai_msg.text()

    # Run a simple q&a session
    i=0
    while True:
        user_input = input(f"Q{i+1}: ")   # prompt user in Jupyter cell
        if user_input == "quit" or user_input == "exit" or user_input == "break" or not user_input.strip():        # allow empty string or quit or exit or break to break early
            break
        answer = chat_turn(user_input)
        i += 1
        print(f"Assistant: {answer}\n")
        print({answer})

        # Example questions are: where were the last olympics held?  who won the world cup in 2022?  Who wa the team captain? what is the capital of france? who is the president of the united states? what is the tallest mountain in the world?
        # who won the women's 100m  hurldes at the 2025 tokyo world athletics championship

Q1: who is the president of the USA?
Assistant: As of September 22, 2025, the President of the United States is Donald Trump. He was inaugurated as the 47th president on January 20, 2025, after defeating Kamala Harris in the 2024 presidential election. This marks his second term in office, having previously served as the 45th president from 2017 to 2021. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Second_presidency_of_Donald_Trump?utm_source=openai))

In the 2024 election, Trump, a Republican, faced Harris, the Democratic nominee and then-Vice President under President Joe Biden. Trump's victory in the 2024 election led to his re-inauguration in January 2025. ([apnews.com](https://apnews.com/article/d69ed8609cbb7c38c1f0fe1e074027f2?utm_source=openai))

During his first term, Trump faced significant challenges, including impeachments, criminal indictments, and assassination attempts. Despite these obstacles, he secured re-election in 2024, overcoming a competitive race against Har

In [ ]:
print(history)

[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={})]


In [15]:
from langchain_openai import ChatOpenAI
from pydantic import BaseModel


def get_weather(location: str) -> None:
    """Get weather at a location."""
    return "It's sunny."


class OutputSchema(BaseModel):
    """Schema for response."""

    answer: str
    justification: str


#llm = ChatOpenAI(model="gpt-4.1")

structured_llm = llm.bind_tools(
    [get_weather],
    response_format=OutputSchema,
    strict=True,
)

# Response contains tool calls:
tool_call_response = structured_llm.invoke("What is the weather in SF?")

# structured_response.additional_kwargs["parsed"] contains parsed output
structured_response = structured_llm.invoke(
    "What weighs more, a pound of feathers or a pound of gold?"
)
print(structured_response.content)

{"answer":"A pound of feathers and a pound of gold weigh the same, as both are one pound.","justification":"Weight is a measure of mass, and one pound is equal to one pound regardless of the material being weighed. Therefore, a pound of feathers and a pound of gold both weigh exactly one pound."}


In [ ]:
# Example of a tool call
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the weather for a city",
            "parameters": {
                "type": "object",
                "properties": {"city": {"type": "string"}},
                "required": ["city"],
            },
        },
    }
]

resp = client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[{"role": "user", "content": "What's the weather in London?"}],
    tools=tools,
    tool_choice="auto",
)

print(resp.choices[0].message)
